In [7]:
import requests
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import os
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping
from datetime import datetime, timedelta
import pickle
import warnings

warnings.filterwarnings('ignore')

# Load environment variables
load_dotenv()
API_KEY = os.getenv("API_KEY")
BASE_URL = "https://api.openweathermap.org/data/3.0/onecall/timemachine"  # Check the correct endpoint

# Set up date range and city details
Current_Date = datetime.now().date()
Start_Date = (Current_Date - timedelta(days=3 * 365))
Cities_Details = {
    "Patna": {
        "State/UT": "Bihar",
        "Latitude": 25.5941,
        "Longitude": 85.1376,
    },
    "National Capital Region": {
        "State/UT": "Delhi, Haryana, Rajasthan, Uttar Pradesh",
        "Latitude": 28.6139,
        "Longitude": 77.2090,
    },
    "Mumbai": {
        "State/UT": "Maharashtra",
        "Latitude": 19.0760,
        "Longitude": 72.8777,
    },
    "Kolkata": {
        "State/UT": "West Bengal",
        "Latitude": 22.5726,
        "Longitude": 88.3639,
    },
    "Chennai": {
        "State/UT": "Tamil Nadu",
        "Latitude": 13.0827,
        "Longitude": 80.2707,
    },
    "Bangalore": {
        "State/UT": "Karnataka",
        "Latitude": 12.9716,
        "Longitude": 77.5946,
    },
    "Hyderabad": {
        "State/UT": "Telangana",
        "Latitude": 17.3850,
        "Longitude": 78.4867,
    },
    "Pune": {
        "State/UT": "Maharashtra",
        "Latitude": 18.5204,
        "Longitude": 73.8567,
    },
    "Andhra Pradesh Capital Region": {
        "State/UT": "Andhra Pradesh",
        "Latitude": 16.5062,
        "Longitude": 80.6480,
    },
    "Ahmedabad": {
        "State/UT": "Gujarat",
        "Latitude": 23.0225,
        "Longitude": 72.5714,
    },
    "Surat": {
        "State/UT": "Gujarat",
        "Latitude": 21.1702,
        "Longitude": 72.8311,
    },
    "Visakhapatnam": {
        "State/UT": "Andhra Pradesh",
        "Latitude": 17.6868,
        "Longitude": 83.2185,
    },
    "Jaipur": {
        "State/UT": "Rajasthan",
        "Latitude": 26.9124,
        "Longitude": 75.7873,
    },
    "Lucknow": {
        "State/UT": "Uttar Pradesh",
        "Latitude": 26.8467,
        "Longitude": 80.9462,
    },
    "Kanpur": {
        "State/UT": "Uttar Pradesh",
        "Latitude": 26.4499,
        "Longitude": 80.3319,
    }
}
city_name = 'Mumbai'
LAT = Cities_Details[city_name]['Latitude']
LON = Cities_Details[city_name]['Longitude']

weather_data = {
    'date': [],
    'temp_min': [],
    'temp_max': [],
    'humidity': [],
    'wind_speed': []
}

# Loop through the dates
current_date = Start_Date
while current_date <= Current_Date:
    date_str = int(datetime.combine(current_date, datetime.min.time()).timestamp())  # Convert date to timestamp
    url = f"{BASE_URL}?lat={LAT}&lon={LON}&dt={date_str}&appid=6b33a74b8dd94a967b2622a5eb6c1d93&units=metric"   
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()      
        # Access the first day's data
        daily_data = data['data'][0]  # Get the first day's data
        min_temp = daily_data['temp']  # You might need to adjust this if there's no specific 'min' or 'max'
        max_temp = daily_data['temp']  # Assuming 'temp' gives the current temp; adjust as necessary
        humidity = daily_data['humidity']
        wind_speed = daily_data['wind_speed']

        weather_data['date'].append(current_date)
        weather_data['temp_min'].append(min_temp)  # If min_temp is not separately provided, you might set this to another value
        weather_data['temp_max'].append(max_temp)  # Same here for max_temp
        weather_data['humidity'].append(humidity)
        weather_data['wind_speed'].append(wind_speed)
    else:
        print(f"Failed to fetch data for {current_date}: {response.status_code}")

    # Move to the next date
    current_date += timedelta(days=1)

# Create a DataFrame from the collected data
df = pd.DataFrame(weather_data)

# Check if df is not empty before proceeding
if not df.empty:
    # Sort DataFrame by date
    df = df.sort_values('date')
    
    # Compute daily average temperature
    df['temp_avg'] = (df['temp_min'] + df['temp_max']) / 2
    
    # Define features and target variable
    features = ['temp_avg', 'humidity', 'wind_speed']
    target = 'temp_avg'
    
    # Normalize features
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(df[features])
    
    # Prepare X and y
    X, y = [], []
    
    # Use the previous 7 days to predict the next day's temperature
    for i in range(7, len(scaled_data)):
        X.append(scaled_data[i-7:i])
        y.append(scaled_data[i][0])  # temp_avg is the target variable
    
    # Convert lists to numpy arrays
    X, y = np.array(X), np.array(y)

    X_train, X_test = X[:-5], X[-5:]
    y_train, y_test = y[:-5], y[-5:]

    # Build the LSTM model
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(LSTM(units=50, return_sequences=False))
    model.add(Dense(units=25))
    model.add(Dense(units=1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    model.fit(X_train, y_train, epochs=100, batch_size=1, validation_split=0.2, callbacks=[early_stopping], verbose=0)

    # Create the 'artifacts' directory if it doesn't exist
    artifacts_dir = "artifacts"
    if not os.path.exists(artifacts_dir):
        os.makedirs(artifacts_dir)

    # Save the model
    model.save(os.path.join(artifacts_dir, f"{city_name}_model.h5"))

    # Save the scaler
    with open(os.path.join(artifacts_dir, f"{city_name}_scaler.pkl"), 'wb') as f:
        pickle.dump(scaler, f)
else:
    print("No weather data collected.")